## This function measures duration, pitch, HNR, jitter, and shimmer

# Measure Pitch, HNR, Jitter, Shimmer, Formants, and Estimate VTL

## Import the external modules

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:

!pip install praat-parselmouth
!pip install essentia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [55]:

#!/usr/bin/env python3
import glob
import numpy as np
import pandas as pd
import parselmouth 
import statistics


from parselmouth.praat import call
from scipy.stats.mstats import zscore
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from essentia import standard as ess
import librosa

In [56]:
# This is the function to measure source acoustics using default male parameters.

def measurePitch(voiceID, f0min, f0max, unit):
    sound = parselmouth.Sound(voiceID) # read the sound
    duration = call(sound, "Get total duration") # duration
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max) #create a praat pitch object
    meanF0 = call(pitch, "Get mean", 0, 0, unit) # get mean pitch
    stdevF0 = call(pitch, "Get standard deviation", 0 ,0, unit) # get standard deviation
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, f0min, 0.1, 1.0)
    hnr = call(harmonicity, "Get mean", 0, 0)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)
    rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
    ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)
    ddpJitter = call(pointProcess, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3)
    localShimmer =  call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    localdbShimmer = call([sound, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq3Shimmer = call([sound, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    aqpq5Shimmer = call([sound, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq11Shimmer =  call([sound, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    ddaShimmer = call([sound, pointProcess], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    
    return duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer

## This function measures formants at each glottal pulse

Puts, D. A., Apicella, C. L., & Cárdenas, R. A. (2012). Masculine voices signal men's threat potential in forager and industrial societies. Proceedings of the Royal Society of London B: Biological Sciences, 279(1728), 601-609.

Adapted from: DOI 10.17605/OSF.IO/K2BHS

In [57]:
# This function measures formants using Formant Position formula
def measureFormants(sound, wave_file, f0min,f0max):
    sound = parselmouth.Sound(sound) # read the sound
    pitch = call(sound, "To Pitch (cc)", 0, f0min, 15, 'no', 0.03, 0.45, 0.01, 0.35, 0.14, f0max)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    
    formants = call(sound, "To Formant (burg)", 0.0025, 5, 5000, 0.025, 50)
    numPoints = call(pointProcess, "Get number of points")

    f1_list = []
    f2_list = []
    f3_list = []
    f4_list = []
    
    # Measure formants only at glottal pulses
    for point in range(0, numPoints):
        point += 1
        t = call(pointProcess, "Get time from index", point)
        f1 = call(formants, "Get value at time", 1, t, 'Hertz', 'Linear')
        f2 = call(formants, "Get value at time", 2, t, 'Hertz', 'Linear')
        f3 = call(formants, "Get value at time", 3, t, 'Hertz', 'Linear')
        f4 = call(formants, "Get value at time", 4, t, 'Hertz', 'Linear')
        f1_list.append(f1)
        f2_list.append(f2)
        f3_list.append(f3)
        f4_list.append(f4)
    
    f1_list = [f1 for f1 in f1_list if str(f1) != 'nan']
    f2_list = [f2 for f2 in f2_list if str(f2) != 'nan']
    f3_list = [f3 for f3 in f3_list if str(f3) != 'nan']
    f4_list = [f4 for f4 in f4_list if str(f4) != 'nan']
    
    # calculate mean formants across pulses
    f1_mean = statistics.mean(f1_list)
    f2_mean = statistics.mean(f2_list)
    f3_mean = statistics.mean(f3_list)
    f4_mean = statistics.mean(f4_list)
    
    # calculate median formants across pulses, this is what is used in all subsequent calcualtions
    # you can use mean if you want, just edit the code in the boxes below to replace median with mean
    f1_median = statistics.median(f1_list)
    f2_median = statistics.median(f2_list)
    f3_median = statistics.median(f3_list)
    f4_median = statistics.median(f4_list)
    
    return f1_mean, f2_mean, f3_mean, f4_mean, f1_median, f2_median, f3_median, f4_median


In [58]:
def calculate_mfcc(sound):
  mfcc_0, mfcc_1, mfcc_2, mfcc_3, mfcc_4, mfcc_5, mfcc_6, mfcc_7, mfcc_8, mfcc_9, mfcc_10, mfcc_11, mfcc_12 = 0, 0, 0,0,0,0,0,0,0,0,0,0, 0
  hamming_window = ess.Windowing(type='hamming')
  spectrum = ess.Spectrum()  # we just want the magnitude spectrum
  mfcc = ess.MFCC(numberCoefficients=13)
  frame_sz = 1024
  hop_sz = 500

  mfccs = np.array([mfcc(spectrum(hamming_window(frame)))[1]
                for frame in ess.FrameGenerator(sound, frameSize=frame_sz, hopSize=hop_sz)])
  print(mfccs.shape)
  mfcc_0 = mfccs[1][0]
  mfcc_1 = mfccs[1][1]
  mfcc_2 = mfccs[1][2]
  mfcc_3 = mfccs[1][3]
  mfcc_4 = mfccs[1][4]
  mfcc_5 = mfccs[1][5]
  mfcc_6 = mfccs[1][6]
  mfcc_7 = mfccs[1][7]
  mfcc_8 = mfccs[1][8]
  mfcc_9 = mfccs[1][9]
  mfcc_10 = mfccs[1][10]
  mfcc_11 = mfccs[1][11]
  mfcc_12 = mfccs[1][12]
  return mfcc_0, mfcc_1, mfcc_2, mfcc_3, mfcc_4, mfcc_5, mfcc_6, mfcc_7, mfcc_8, mfcc_9, mfcc_10, mfcc_11, mfcc_12


## This function runs a 2-factor Principle Components Analysis (PCA) on Jitter and Shimmer

In [59]:
def runPCA(df):
    # z-score the Jitter and Shimmer measurements
    measures = ['localJitter', 'localabsoluteJitter', 'rapJitter', 'ppq5Jitter', 'ddpJitter',
                'localShimmer', 'localdbShimmer', 'apq3Shimmer', 'apq5Shimmer', 'apq11Shimmer', 'ddaShimmer']
    x = df.loc[:, measures].values
    x = StandardScaler().fit_transform(x)
    # PCA
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data = principalComponents, columns = ['JitterPCA', 'ShimmerPCA'])
    principalDf
    return principalDf

## This block of code runs the above functions on all of the '.wav' files in the /audio folder

In [60]:
# create lists to put the results
file_list = []
duration_list = []
mean_F0_list = []
sd_F0_list = []
hnr_list = []
localJitter_list = []
localabsoluteJitter_list = []
rapJitter_list = []
ppq5Jitter_list = []
ddpJitter_list = []
localShimmer_list = []
localdbShimmer_list = []
apq3Shimmer_list = []
aqpq5Shimmer_list = []
apq11Shimmer_list = []
ddaShimmer_list = []
f1_mean_list = []
f2_mean_list = []
f3_mean_list = []
f4_mean_list = []
f1_median_list = []
f2_median_list = []
f3_median_list = []
f4_median_list = []
mfcc_0 = []
mfcc_1 = []
mfcc_2 = []
mfcc_3 = []
mfcc_4 = []
mfcc_5 = []
mfcc_6 = []
mfcc_7 = []
mfcc_8 = []
mfcc_9 = []
mfcc_10 = []
mfcc_11 = []
mfcc_12 = []

# Go through all the wave files in the folder and measure all the acoustics
for wave_file in glob.glob("/content/drive/MyDrive/Spl topics in AI/Project/26-29_09_2017_KCL/ReadText/HC/*.wav"):
    sound = parselmouth.Sound(wave_file)
    y, fs = librosa.load(wave_file)
    (duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, 
     localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer) = measurePitch(
        sound, 75, 300, "Hertz")
    (f1_mean, f2_mean, f3_mean, f4_mean, f1_median, f2_median, f3_median, f4_median) = measureFormants(
        sound, wave_file, 75, 300)
    (m0, m1, m2, m3, m4, m5, m6, m7, m8, m9, m10, m11, m12) = calculate_mfcc(y)
    file_list.append(wave_file) # make an ID list
    duration_list.append(duration) # make duration list
    mean_F0_list.append(meanF0) # make a mean F0 list
    sd_F0_list.append(stdevF0) # make a sd F0 list
    hnr_list.append(hnr) #add HNR data
    
    # add raw jitter and shimmer measures
    localJitter_list.append(localJitter)
    localabsoluteJitter_list.append(localabsoluteJitter)
    rapJitter_list.append(rapJitter)
    ppq5Jitter_list.append(ppq5Jitter)
    ddpJitter_list.append(ddpJitter)
    localShimmer_list.append(localShimmer)
    localdbShimmer_list.append(localdbShimmer)
    apq3Shimmer_list.append(apq3Shimmer)
    aqpq5Shimmer_list.append(aqpq5Shimmer)
    apq11Shimmer_list.append(apq11Shimmer)
    ddaShimmer_list.append(ddaShimmer)
    
    # add the formant data
    f1_mean_list.append(f1_mean)
    f2_mean_list.append(f2_mean)
    f3_mean_list.append(f3_mean)
    f4_mean_list.append(f4_mean)
    f1_median_list.append(f1_median)
    f2_median_list.append(f2_median)
    f3_median_list.append(f3_median)
    f4_median_list.append(f4_median)

    #add mfcc data
    mfcc_0.append(m0)
    mfcc_1.append(m1)
    mfcc_2.append(m2)
    mfcc_3.append(m3)
    mfcc_4.append(m4)
    mfcc_5.append(m5)
    mfcc_6.append(m6)
    mfcc_7.append(m7)
    mfcc_8.append(m8)
    mfcc_9.append(m9)
    mfcc_10.append(m10)
    mfcc_11.append(m11)
    mfcc_12.append(m12)

(6666, 13)
(7239, 13)
(6156, 13)
(4893, 13)
(6445, 13)
(5535, 13)
(6033, 13)
(6545, 13)
(5758, 13)
(7429, 13)
(7526, 13)
(6231, 13)
(5391, 13)
(7386, 13)
(8980, 13)
(7690, 13)
(6781, 13)
(6431, 13)
(8512, 13)
(4113, 13)
(6996, 13)


## This block of code adds all of that data we just generated to a Pandas data frame

In [61]:
# Add the data to Pandas
df = pd.DataFrame(np.column_stack([file_list, duration_list, mean_F0_list, sd_F0_list, hnr_list, 
                                   localJitter_list, localabsoluteJitter_list, rapJitter_list, 
                                   ppq5Jitter_list, ddpJitter_list, localShimmer_list, 
                                   localdbShimmer_list, apq3Shimmer_list, aqpq5Shimmer_list, 
                                   apq11Shimmer_list, ddaShimmer_list, f1_mean_list, 
                                   f2_mean_list, f3_mean_list, f4_mean_list, 
                                   f1_median_list, f2_median_list, f3_median_list, 
                                   f4_median_list, mfcc_0, mfcc_1, mfcc_2, mfcc_3, mfcc_4, 
                                   mfcc_5, mfcc_6, mfcc_7, mfcc_8, mfcc_9, mfcc_10, mfcc_11, 
                                   mfcc_12]),
                                   columns=['voiceID', 'duration', 'meanF0Hz', 'stdevF0Hz', 'HNR', 
                                            'localJitter', 'localabsoluteJitter', 'rapJitter', 
                                            'ppq5Jitter', 'ddpJitter', 'localShimmer', 
                                            'localdbShimmer', 'apq3Shimmer', 'apq5Shimmer', 
                                            'apq11Shimmer', 'ddaShimmer', 'f1_mean', 'f2_mean', 
                                            'f3_mean', 'f4_mean', 'f1_median', 
                                            'f2_median', 'f3_median', 'f4_median', 'mfcc_coeff_0',
                                            'mfcc_coeff_1','mfcc_coeff_2','mfcc_coeff_3','mfcc_coeff_4',
                                            'mfcc_coeff_5','mfcc_coeff_6','mfcc_coeff_7','mfcc_coeff_8',
                                            'mfcc_coeff_9','mfcc_coeff_10', 'mfcc_coeff_11', 'mfcc_coeff_12'])

pcaData = runPCA(df) # Run jitter and shimmer PCA
df = pd.concat([df, pcaData], axis=1) # Add PCA data
# reload the data so it's all numbers
df.to_csv("processed_results.csv", index=False)
df = pd.read_csv('processed_results.csv', header=0)
df.sort_values('voiceID').head(20)

,voiceID,duration,meanF0Hz,stdevF0Hz,HNR,localJitter,localabsoluteJitter,rapJitter,ppq5Jitter,ddpJitter,...,mfcc_coeff_5,mfcc_coeff_6,mfcc_coeff_7,mfcc_coeff_8,mfcc_coeff_9,mfcc_coeff_10,mfcc_coeff_11,mfcc_coeff_12,JitterPCA,ShimmerPCA
0,/content/drive/MyDrive/Spl topics in AI/Projec...,151.114717,173.028992,26.042716,13.292053,0.026783,0.000155,0.011867,0.013015,0.035600,...,0.514710,10.263039,12.900982,2.136227,-4.437233,-7.277573,-10.454300,-7.892563,4.254054,1.280083
1,/content/drive/MyDrive/Spl topics in AI/Projec...,164.105057,188.803016,29.907306,11.243993,0.015988,0.000085,0.006635,0.006978,0.019906,...,-2.016930,7.363617,8.214668,2.258114,-5.973297,-7.904732,-1.257801,5.816917,-3.150761,0.125995
2,/content/drive/MyDrive/Spl topics in AI/Projec...,139.555329,123.982623,23.363294,13.423983,0.026935,0.000218,0.010964,0.011671,0.032892,...,11.709869,7.403202,0.655388,11.021908,-6.382938,-15.637440,-16.191143,-3.128807,3.047446,0.350653
3,/content/drive/MyDrive/Spl topics in AI/Projec...,110.926916,180.520417,34.073202,12.235210,0.020818,0.000115,0.008991,0.009034,0.026972,...,-17.807377,-6.861572,-0.492939,-2.493622,3.548691,4.959911,1.983826,2.056862,-1.665359,-0.676599
4,/content/drive/MyDrive/Spl topics in AI/Projec...,146.104762,194.261126,37.678373,14.669165,0.015138,0.000078,0.005401,0.005984,0.016203,...,2.465500,-28.020630,-14.496765,-6.295643,-1.954392,-4.320580,4.340382,-13.062920,-2.736372,1.659618
5,/content/drive/MyDrive/Spl topics in AI/Projec...,125.475986,166.086083,29.986251,13.020618,0.021731,0.000131,0.009236,0.009851,0.027708,...,9.321476,-6.753918,-17.205510,-46.151703,-28.755314,-21.740276,-22.081978,9.024506,-1.585232,-1.077452
6,/content/drive/MyDrive/Spl topics in AI/Projec...,136.772336,194.425402,35.712023,12.333993,0.026983,0.000138,0.012409,0.012859,0.037228,...,-21.050861,-10.241852,-1.168648,-6.946926,4.972511,5.923584,-12.082623,-9.059074,2.592863,-0.382771
7,/content/drive/MyDrive/Spl topics in AI/Projec...,148.386304,195.011773,30.814328,15.683277,0.017478,0.000090,0.007061,0.007589,0.021182,...,-0.531769,2.721527,3.151360,3.300243,0.611279,-3.214226,-6.539284,-7.426819,-2.655162,0.054695
8,/content/drive/MyDrive/Spl topics in AI/Projec...,130.531247,196.924965,36.976910,11.784297,0.024897,0.000126,0.011305,0.011848,0.033914,...,-11.939983,-7.770908,3.728642,3.629745,-2.396542,-23.845087,-15.211826,-13.916004,1.381485,-0.187240
9,/content/drive/MyDrive/Spl topics in AI/Projec...,168.417664,155.210061,40.396894,13.388906,0.034058,0.000219,0.017490,0.016571,0.052470,...,-6.118233,4.170887,23.894123,-3.409153,-2.590775,12.718216,18.345272,1.083084,6.918827,-1.755503


## Next we calculate the vocal-tract length estimates

### Formant position
 Puts, D. A., Apicella, C. L., & Cárdenas, R. A. (2012). Masculine voices signal men's threat potential in forager and industrial societies. Proceedings of the Royal Society of London B: Biological Sciences, 279(1728), 601-609.

In [62]:
df['pF'] = (zscore(df.f1_median) + zscore(df.f2_median) + zscore(df.f3_median) + zscore(df.f4_median)) / 4

### Formant Dispersion
Fitch, W. T. (1997). Vocal tract length and formant frequency dispersion correlate with body size in rhesus macaques. The Journal of the Acoustical Society of America, 102(2), 1213-1222.

In [63]:
df['fdisp'] = (df['f4_median'] - df['f1_median']) / 3

### Fn (Average Formant)
Pisanski, K., & Rendall, D. (2011). The prioritization of voice fundamental frequency or formants in listeners’ assessments of speaker size, masculinity, and attractiveness. The Journal of the Acoustical Society of America, 129(4), 2201-2212.

In [64]:
df['avgFormant'] = (df['f1_median'] + df['f2_median'] + df['f3_median'] + df['f4_median']) / 4

### MFF 
Smith, D. R., & Patterson, R. D. (2005). The interaction of glottal-pulse rate and vocal-tract length in judgements of speaker size, sex, and age. The Journal of the Acoustical Society of America, 118(5), 3177-3186.

In [65]:
df['mff'] = (df['f1_median'] * df['f2_median'] * df['f3_median'] * df['f4_median']) ** 0.25

### Fitch VTL
Fitch, W. T. (1997). Vocal tract length and formant frequency dispersion correlate with body size in rhesus macaques. The Journal of the Acoustical Society of America, 102(2), 1213-1222.

In [66]:
# reload the data again
df.to_csv("processed_results.csv", index=False)
df = pd.read_csv('processed_results.csv', header=0)

df['fitch_vtl'] = ((1 * (35000 / (4 * df['f1_median']))) +
                   (3 * (35000 / (4 * df['f2_median']))) + 
                   (5 * (35000 / (4 * df['f3_median']))) + 
                   (7 * (35000 / (4 * df['f4_median'])))) / 4

### $\Delta$F 
Reby,D.,& McComb,K.(2003). Anatomical constraints generate honesty: acoustic cues to age and weight in the roars of red deer stags. Animal Behaviour, 65, 519e-530.

In [67]:
xysum = (0.5 * df['f1_median']) + (1.5 * df['f2_median']) + (2.5 * df['f3_median']) + (3.5 * df['f4_median'])
xsquaredsum = (0.5 ** 2) + (1.5 ** 2) + (2.5 ** 2) + (3.5 ** 2)
df['delta_f'] = xysum / xsquaredsum

### VTL($\Delta$F)
Reby,D.,&McComb,K.(2003).Anatomical constraints generate honesty: acoustic cues to age and weight in the roars of red deer stags. Animal Behaviour, 65, 519e-530.

In [68]:
df['vtl_delta_f'] = 35000 / (2 * df['delta_f'])

## Save the final data

In [69]:
# Write out the final dataframe
df.to_csv("processed_results2_hc.csv", index=False)

## Run this to tell you when it's done